# Join the dataframes

In [1]:
import pandas as pd 

In [3]:
from codecs import utf_8_encode


suburb = pd.read_csv("../data/raw/sub_4_pred.csv")
school = pd.read_csv("../data/raw/dv331_schoollocations2022.csv", encoding='cp1252')

In [10]:
school

,Education_Sector,Entity_Type,SCHOOL_NO,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.66660,-38.61771
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.59039,-38.38628
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.47565,-37.08450
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.23472,-36.90137
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.21398,-37.74268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2294,Independent,2,2243,Lysterfield Lake College,Primary,O,20 A’Beckett Road,NaN,NARRE WARREN NORTH,VIC,...,20 A’Beckett Road,NaN,NARRE WARREN NORTH,VIC,3804,03-,161,Casey (C),145.31604,-37.97748
2295,Independent,2,2244,Plenty River College,Secondary,O,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,...,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,3752,03 9435 9060,707,Whittlesea (C),145.08148,-37.64875
2296,Catholic,2,2245,Holy Cross Catholic Primary School,Primary,O,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,...,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,3064,03 9216 4200,327,Hume (C),144.90520,-37.53046
2297,Independent,2,2246,Sidrah Gardens School,Primary,O,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,...,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,3804,03-,161,Casey (C),145.31589,-37.97324


In [22]:
school_count = school[["School_Name", "Postal_Postcode"]].groupby(by="Postal_Postcode").count()
school_count

,School_Name
Postal_Postcode,
3000,4
3003,1
3004,4
3006,2
3008,1
...,...
3987,1
3988,1
3992,1


In [20]:
merge = school_count.merge(suburb, right_on="Postal_Postcode", left_on="Postcode")

AttributeError: 'Series' object has no attribute 'merge'